In [1]:
!wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE72nnn/GSE72267/matrix/GSE72267_series_matrix.txt.gz

--2026-02-04 08:34:25--  https://ftp.ncbi.nlm.nih.gov/geo/series/GSE72nnn/GSE72267/matrix/GSE72267_series_matrix.txt.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.13, 130.14.250.12, 130.14.250.31, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7252003 (6.9M) [application/x-gzip]
Saving to: ‘GSE72267_series_matrix.txt.gz’

GSE72267_series_mat 100%[===================>]   6.92M  3.05MB/s    in 2.3s    

2026-02-04 08:34:29 (3.05 MB/s) - ‘GSE72267_series_matrix.txt.gz’ saved [7252003/7252003]



In [10]:
import pandas as pd
import gzip
from io import StringIO


In [11]:
file_path = "/workspaces/Reasearch_Genomic/GSE72267_series_matrix.txt.gz"

with gzip.open(file_path, 'rt') as f:
    lines = f.readlines()


In [12]:
start = lines.index("!series_matrix_table_begin\n") + 1
end = lines.index("!series_matrix_table_end\n")

data = "".join(lines[start:end])
df = pd.read_csv(StringIO(data), sep="\t")


In [13]:
df

,ID_REF,GSM1859079,GSM1859080,GSM1859081,GSM1859082,GSM1859083,GSM1859084,GSM1859085,GSM1859086,GSM1859087,...,GSM1859128,GSM1859129,GSM1859130,GSM1859131,GSM1859132,GSM1859133,GSM1859134,GSM1859135,GSM1859136,GSM1859137
0,1007_s_at,7.349298,7.274405,6.719570,7.083267,7.452170,7.072315,7.051066,6.915191,6.724721,...,7.220690,7.273062,6.348098,7.215291,7.158651,6.939362,6.953196,6.783862,7.508211,7.442646
1,1053_at,8.906082,8.997323,9.145367,8.947209,8.679888,9.021920,8.833503,8.996459,8.499686,...,9.114221,8.149797,9.125882,8.945029,9.098237,8.231308,8.962738,8.335829,8.669423,9.209691
2,117_at,10.916643,11.173430,11.369026,11.274212,10.783615,11.126310,10.577591,11.064433,10.878215,...,11.873479,11.157583,11.803321,11.393424,11.568479,11.679601,11.522308,11.012502,11.605217,12.168853
3,121_at,6.954244,6.334305,6.733312,7.096500,6.813166,6.610517,6.888822,6.995565,6.561855,...,6.646920,6.713738,7.034526,6.822774,6.696431,6.798336,6.599626,6.271728,5.885947,6.800522
4,1255_g_at,2.702785,2.581031,2.469741,2.774409,2.642078,2.732572,2.646264,3.032687,2.642967,...,2.782319,2.540955,2.672644,2.703629,2.577448,2.678776,2.649259,2.667041,2.651123,2.543898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22272,AFFX-ThrX-5_at,3.086941,3.395082,3.211710,3.177194,3.240693,3.428652,3.414479,3.639821,3.213977,...,3.291512,3.084009,3.300177,3.388613,3.267444,3.089564,3.082268,3.087934,3.056692,3.401986
22273,AFFX-ThrX-M_at,3.073432,3.257221,3.403392,3.409128,3.037217,3.126156,3.325499,3.309700,3.181855,...,3.364188,3.175478,3.262015,3.099387,3.411084,3.338604,3.289487,3.344057,3.080960,3.040670
22274,AFFX-TrpnX-3_at,2.467247,2.692106,2.633840,2.517776,2.546762,2.815358,2.608742,3.069019,2.621296,...,2.775582,2.702807,2.518298,2.778495,2.682865,2.666095,2.819289,2.741852,2.770820,2.730419
22275,AFFX-TrpnX-5_at,3.531346,3.475607,3.624394,3.510254,3.376297,3.778116,3.334487,3.659660,3.388264,...,3.582253,3.554952,3.515270,3.853754,3.298534,3.684110,3.390299,3.412506,3.631528,3.421109


In [14]:
df = df.set_index(df.columns[0])
X = df.T        # samples as rows


In [23]:
import numpy as np

y = np.random.randint(0, 2, size=X_scaled.shape[0])



In [24]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [25]:
from sklearn.linear_model import LassoCV

lasso = LassoCV(
    cv=5,
    max_iter=5000,
    n_jobs=-1,
    random_state=42
)

lasso.fit(X_scaled, y)


,"eps eps: float, default=1e-3Length of the path. ``eps=1e-3`` means that``alpha_min / alpha_max = 1e-3``.",0.001
,"n_alphas n_alphas: int, default=100Number of alphas along the regularization path... deprecated:: 1.7 `n_alphas` was deprecated in 1.7 and will be removed in 1.9. Use `alphas` instead.",'deprecated'
,"alphas alphas: array-like or int, default=NoneValues of alphas to test along the regularization path.If int, `alphas` values are generated automatically.If array-like, list of alpha values to use... versionchanged:: 1.7 `alphas` accepts an integer value which removes the need to pass `n_alphas`... deprecated:: 1.7 `alphas=None` was deprecated in 1.7 and will be removed in 1.9, at which point the default value will be set to 100.",'warn'
,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto false, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"precompute precompute: 'auto', bool or array-like of shape (n_features, n_features), default='auto'Whether to use a precomputed Gram matrix to speed upcalculations. If set to ``'auto'`` let us decide. The Grammatrix can also be passed as argument.",'auto'
,"max_iter max_iter: int, default=1000The maximum number of iterations.",5000
,"tol tol: float, default=1e-4The tolerance for the optimization: if the updates are smaller or equal to``tol``, the optimization code checks the dual gap for optimality and continuesuntil it is smaller or equal to ``tol``.",0.0001
,"copy_X copy_X: bool, default=TrueIf ``True``, X will be copied; else, it may be overwritten.",True
,"cv cv: int, cross-validation generator or iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross-validation,- int, to specify the number of folds.- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For int/None inputs, :class:`~sklearn.model_selection.KFold` is used.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: bool or int, default=FalseAmount of verbosity.",False
,"n_jobs n_jobs: int, default=NoneNumber of CPUs to use during the cross validation.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",-1


In [26]:
print("Best alpha:", lasso.alpha_)


Best alpha: 0.2528868107106251


In [27]:
import numpy as np

coef_df = pd.DataFrame({
    "Gene": X.columns,
    "Coefficient": lasso.coef_
})

top32 = coef_df.reindex(
    coef_df.Coefficient.abs().sort_values(ascending=False).index
).head(32)

top32


,Gene,Coefficient
0,1007_s_at,0.0
1,1053_at,0.0
2,117_at,0.0
3,121_at,0.0
4,1255_g_at,0.0
5,1294_at,0.0
6,1316_at,0.0
7,1320_at,0.0
8,1405_i_at,0.0
9,1431_at,0.0


In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [29]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [30]:
lasso_clf = LogisticRegression(
    penalty="l1",
    solver="saga",
    C=0.1,          # inverse of regularization strength
    max_iter=5000,
    n_jobs=-1
)

lasso_clf.fit(X_scaled, y)


/usr/local/python/3.12.1/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1160: UserWarning: Inconsistent values: penalty=l1 with l1_ratio=0.0. penalty is deprecated. Please use l1_ratio only.
  warnings.warn(
/usr/local/python/3.12.1/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1184: FutureWarning: 'n_jobs' has no effect since 1.8 and will be removed in 1.10. You provided 'n_jobs=-1', please leave it unspecified.
  warnings.warn(msg, category=FutureWarning)


,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'l1'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",0.1
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`multiclass`

In [31]:
coef_df = pd.DataFrame({
    "Gene": X.columns,
    "Coefficient": lasso_clf.coef_[0]
})

top32 = coef_df.reindex(
    coef_df.Coefficient.abs().sort_values(ascending=False).index
).head(32)

top32


,Gene,Coefficient
4106,204580_at,-0.160551
20420,221058_s_at,-0.148842
7235,207715_at,-0.099405
20362,220999_s_at,0.052358
18918,219555_s_at,-0.051095
8080,208583_x_at,-0.049875
11018,211614_at,-0.044071
17326,217962_at,-0.031426
18467,219104_at,-0.030466
3461,203935_at,0.017667


In [32]:
top32.to_csv("GSE72267_Top32_Lasso_Genes.csv", index=False)


do to ann and accurancy and loss graph